In [1]:
import numpy as np
import pandas as pd

In [6]:
# load data
df = pd.read_excel('worldcities.xlsx')
df.head()

,ville,ville_ascii,lat,lng,pays,iso2,iso3,admin_nom,capital,population,id
0,A Coruña,A Coruna,43.3667,-8.3833,Spain,ES,ESP,Galicia,minor,245468.0,1.724417e+09
1,A Yun Pa,A Yun Pa,13.3939,108.4408,Vietnam,VN,VNM,Gia Lai,minor,53720.0,1.704946e+09
2,Aabenraa,Aabenraa,55.0444,9.4181,Denmark,DK,DNK,Syddanmark,minor,16401.0,1.208000e+09
3,Aachen,Aachen,50.7756,6.0836,Germany,DE,DEU,North Rhine-Westphalia,minor,249070.0,1.276806e+09
4,Aadorf,Aadorf,47.4939,8.8975,Switzerland,CH,CHE,Thurgau,NaN,9036.0,1.756023e+09


In [7]:
# drop and rename
df = df.drop(['ville_ascii', 'capital', 'id', 'admin_nom'], axis=1)
df.rename({
    'ville': 'city',
    'pays': 'country'
}, axis=1, inplace=True)
df.head()

,city,lat,lng,country,iso2,iso3,population
0,A Coruña,43.3667,-8.3833,Spain,ES,ESP,245468.0
1,A Yun Pa,13.3939,108.4408,Vietnam,VN,VNM,53720.0
2,Aabenraa,55.0444,9.4181,Denmark,DK,DNK,16401.0
3,Aachen,50.7756,6.0836,Germany,DE,DEU,249070.0
4,Aadorf,47.4939,8.8975,Switzerland,CH,CHE,9036.0


In [8]:
# filter data
df = df[df['population'] >= 1000000]
df.shape

(776, 7)

In [9]:
# change type population
df['population'] = df.population.astype('int32')
df.dtypes

city           object
lat           float64
lng           float64
country        object
iso2           object
iso3           object
population      int32
dtype: object

In [10]:
first_df = df.copy()

In [11]:
# remove duplicated and missed values
df = df.drop_duplicates()
df.dropna(thresh=6, inplace=True)
df.shape

(767, 7)

In [12]:
print('number of missings in lat & lng: ', np.logical_or(df.lat.isna(), df.lng.isna()).sum())

number of missings in lat & lng:  3


In [14]:
# fill missing values of 'lat' and 'lng' columns
for coordinate in ['lat', 'lng']:
    for id, ser in df[df[coordinate].isna()].iterrows():
        df.loc[id, coordinate] = df[df['country'] == ser['country']][coordinate].mean()

In [15]:
print('number of missings in lat & lng after filling: ', np.logical_or(df.lat.isna(), df.lng.isna()).sum())

number of missings in lat & lng after filling:  0


In [16]:
df[df['city'] == 'Tehran']

,city,lat,lng,country,iso2,iso3,population
39301,Tehran,35.6892,51.3889,Iran,IR,IRN,14148000


In [17]:
Tehran_lat = 35.6892
Tehran_lng = 51.3889

def haversine_from_teh(lat, lng):
    lamb1 = np.radians(Tehran_lng)
    lamb2 = np.radians(lng)
    phi1 = np.radians(Tehran_lat)
    phi2 = np.radians(lat)
    r = 6371
    a = np.sin((phi2 - phi1) / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin((lamb2 - lamb1) / 2) ** 2
    d = 2 * r * np.atan2(np.sqrt(a), np.sqrt(1 - a))
    return d

In [18]:
# create distance_from_tehran column
df['distance_from_tehran'] = df.apply(lambda row: haversine_from_teh(row.lat, row.lng), axis=1)
df.head()

,city,lat,lng,country,iso2,iso3,population,distance_from_tehran
19,Aba,5.1167,7.3667,Nigeria,NG,NGA,1530000,5634.737243
83,Abidjan,5.3167,-4.0333,Côte d'Ivoire,CI,CIV,4980000,6575.533770
121,Abu Dhabi,24.4667,54.3667,United Arab Emirates,AE,ARE,1483000,1280.150009
131,Abuja,9.0667,7.4833,Nigeria,NG,NGA,3770000,5331.673578
169,Accra,5.5500,-0.2000,Ghana,GH,GHA,2388000,6230.918394


In [19]:
# sort and save dataframe df 
df.sort_values(by=['city', 'lat'], ascending=[True, False], ignore_index=True).to_csv('distances.csv')

In [20]:
# sort and save dataframe first_df
first_df.sort_values(by=['city', 'lat'], ascending=[True, False], ignore_index=True).to_csv('cities.csv')

In [ ]:
import zlib
import zipfile

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

file_names = ['populous_cities.ipynb' , 'cities.csv', 'distances.csv']

compress(file_names)